In [22]:
# @title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License

# Generate Text Embeddings by using Hugging Face Hub models

<table align="left">
  <td>
    <a target="_blank" href="https://colab.sandbox.google.com/github/apache/beam/blob/master/examples/notebooks/beam-ml/data_preprocessing/huggingface_text_embeddings.ipynb.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/apache/beam/blob/master/examples/notebooks/beam-ml/data_preprocessing/huggingface_text_embeddings.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png" />View source on GitHub</a>
  </td>
</table>



## Text Embeddings

Text embeddings are a way of representing text as numerical vectors. This allows computers to understand and process text data, which is essential for many natural language processing (NLP) tasks.

### Uses of text embeddings
By converting text into numerical vectors, text embeddings make it possible for computers to process and analyze text data. This enables a wide range of NLP tasks, including:

* Semantic search: Finding documents or passages that are relevant to a query, even if the query doesn't use the exact same words as the documents.
* Text classification: Categorizng text data into different classes, such as spam or not spam, or positive sentiment or negative sentiment.
* Machine translation: Translating text from one language to another while preserving the meaning.
* Text summarization: Creating shorter summaries of longer pieces of text.

In this notebook, we will use Apache Beam's `MLTransform` to generate embeddings from text data.

Hugging Face's [`SentenceTransformers`](https://huggingface.co/sentence-transformers) framework uses Python to generate sentence, text, and image embeddings.

To generate text embeddings that use Hugging Face models and `MLTransform`, use `SentenceTransformerEmbeddings` to specify the model configuration.

To use `SentenceTransformerEmbeddings`, first install the `the sentence-transformers` package.

# Install dependencies
 Install Apache Beam and the dependencies needed to work with Hugging Face embeddings.

In [ ]:
! git clone https://github.com/apache/beam.git
! cd beam/sdks/python
! pip install beam/sdks/python
! pip install sentence-transformers

In [24]:
import tempfile
import apache_beam as beam
from apache_beam.ml.transforms.base import MLTransform
from apache_beam.ml.transforms.embeddings.huggingface import SentenceTransformerEmbeddings

## Use MLTransform in write mode

In `write` mode, `MLTransform` saves the transforms and their attributes to an artifact location. These transforms are used when you run `MLTransform` in `read` mode.

For more information about using `MLTransform`, see [Preprocess data with MLTransform](https://beam.apache.org/documentation/ml/preprocess-data/) in the Apache Beam documentation.

To generate text embeddings with `MLTransform`, the following pipeline uses the model `sentence-transformers/all-MiniLM-L6-v2` and the text inputs from the Hugging Face blog [Getting Started With Embeddings](https://huggingface.co/blog/getting-started-with-embeddings).

In [25]:
content = [
    {'x': 'How do I get a replacement Medicare card?'},
    {'x': 'What is the monthly premium for Medicare Part B?'},
    {'x': 'How do I terminate my Medicare Part B (medical insurance)?'},
    {'x': 'How do I sign up for Medicare?'},
    {'x': 'Can I sign up for Medicare Part B if I am working and have health insurance through an employer?'},
    {'x': 'How do I sign up for Medicare Part B if I already have Part A?'},
    {'x': 'What are Medicare late enrollment penalties?'},
    {'x': 'What is Medicare and who can get it?'},
    {'x': 'How can I get help with my Medicare Part A and Part B premiums?'},
    {'x': 'What are the different parts of Medicare?'},
    {'x': 'Will my Medicare premiums be higher because of my higher income?'},
    {'x': 'What is TRICARE ?'},
    {'x': "Should I sign up for Medicare Part B if I have Veterans' Benefits?"}
]


# helper function that returns a dict containing only first
#10 elements of generated embeddings.
def truncate_embeddings(d):
  for key in d.keys():
    d[key] = d[key][:10]
  return d

In [26]:
artifact_location_minilm = tempfile.mkdtemp(prefix='huggingface_')
text_embedding_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_transform = SentenceTransformerEmbeddings(
        model_name=text_embedding_model_name, columns=['x'])

with beam.Pipeline() as pipeline:
  data_pcoll = (
          pipeline
          | "CreateData" >> beam.Create(content))
  transformed_pcoll = (
      data_pcoll
      | "MLTransform" >> MLTransform(write_artifact_location=artifact_location_minilm).with_transform(embedding_transform))

  transformed_pcoll | beam.Map(truncate_embeddings) | 'LogOutput' >> beam.Map(print)

  transformed_pcoll | "PrintEmbeddingShape" >> beam.Map(lambda x: print(f"Embedding shape: {len(x['x'])}"))

{'x': [-0.023889463394880295, 0.05525851249694824, -0.011654896661639214, -0.03341428190469742, -0.012260555289685726, -0.024872763082385063, -0.01266342680901289, 0.025345895439386368, 0.01850851997733116, -0.08350814878940582]}
Embedding shape: 10
{'x': [-0.01268761046230793, 0.04687413573265076, -0.010502150282263756, -0.020383981987833977, -0.01336114201694727, 0.04232167452573776, 0.016627851873636246, -0.004099288955330849, -0.0026070312596857548, -0.010187783278524876]}
Embedding shape: 10
{'x': [0.0004943296662531793, 0.11941202729940414, 0.005229473114013672, -0.09273427724838257, 0.007772865705192089, -0.005324989557266235, 0.03450643643736839, -0.05198145657777786, -0.006264965515583754, -0.006110507529228926]}
Embedding shape: 10
{'x': [-0.029711326584219933, 0.02329839952290058, -0.05704096704721451, -0.01218305341899395, -0.013710316270589828, 0.02979600988328457, 0.0637386366724968, 0.0011010386515408754, -0.04512352868914604, -0.040747467428445816]}
Embedding shape: 10


You can also pass additional arguments that are supported by `sentence-transformer` models, such as `convert_to_numpy=False`. These arguments are passed as a `dict` to the `SentenceTransformerEmbeddings` transform by using the `inference_args` parameter.

By passing `convert_to_numpy=False`, the output will contain `torch.Tensor`s.

In [27]:
artifact_location_minilm_with_inference_args = tempfile.mkdtemp(prefix='huggingface_')

embedding_transform = SentenceTransformerEmbeddings(
        model_name=text_embedding_model_name, columns=['x'],
        inference_args={'convert_to_numpy': False}
        )

with beam.Pipeline() as pipeline:
  data_pcoll = (
          pipeline
          | "CreateData" >> beam.Create(content))
  transformed_pcoll = (
      data_pcoll
      | "MLTransform" >> MLTransform(write_artifact_location=artifact_location_minilm_with_inference_args).with_transform(embedding_transform))

  # The outputs are in the Pytorch tensor type.
  transformed_pcoll | 'LogOutput' >> beam.Map(lambda x: print(type(x['x'])))

  transformed_pcoll | "PrintEmbeddingShape" >> beam.Map(lambda x: print(f"Embedding shape: {len(x['x'])}"))


Embedding shape: 384
<class 'torch.Tensor'>
Embedding shape: 384
<class 'torch.Tensor'>
Embedding shape: 384
<class 'torch.Tensor'>
Embedding shape: 384
<class 'torch.Tensor'>
Embedding shape: 384
<class 'torch.Tensor'>
Embedding shape: 384
<class 'torch.Tensor'>
Embedding shape: 384
<class 'torch.Tensor'>
Embedding shape: 384
<class 'torch.Tensor'>
Embedding shape: 384
<class 'torch.Tensor'>
Embedding shape: 384
<class 'torch.Tensor'>
Embedding shape: 384
<class 'torch.Tensor'>
Embedding shape: 384
<class 'torch.Tensor'>
Embedding shape: 384
<class 'torch.Tensor'>


Next, we will use the model `sentence-transformers/sentence-t5-large` to generate text embeddings. The model uses only the encoder from a `T5-large model`. The weights are stored in FP16. For more information about the model, see [Sentence-T5: Scalable Sentence Encoders from Pre-trained Text-to-Text Models](https://arxiv.org/abs/2108.08877).

In [28]:
artifact_location_t5 = tempfile.mkdtemp(prefix='huggingface_t5_')
text_embedding_model_name = 'sentence-transformers/sentence-t5-large'
embedding_transform = SentenceTransformerEmbeddings(
        model_name=text_embedding_model_name, columns=['x'])

with beam.Pipeline() as pipeline:
  data_pcoll = (
          pipeline
          | "CreateData" >> beam.Create(content))
  transformed_pcoll = (
      data_pcoll
      | "MLTransform" >> MLTransform(write_artifact_location=artifact_location_t5).with_transform(embedding_transform))

  transformed_pcoll | beam.Map(truncate_embeddings) | 'LogOutput' >> beam.Map(print)

  transformed_pcoll | "PrintEmbeddingShape" >> beam.Map(lambda x: print(f"Embedding shape: {len(x['x'])}"))

{'x': [-0.0317193828523159, -0.005265399813652039, -0.012499183416366577, 0.00018130357784684747, -0.005592408124357462, 0.06207558885216713, -0.01656288281083107, 0.0167048592120409, -0.01239298190921545, 0.03041897714138031]}
Embedding shape: 10
{'x': [-0.015295305289328098, 0.005405726842582226, -0.015631258487701416, 0.022797023877501488, -0.027843449264764786, 0.03968179598450661, -0.004387892782688141, 0.022909151390194893, 0.01015392318367958, 0.04723235219717026]}
Embedding shape: 10
{'x': [-0.03450256213545799, -0.002632762538269162, -0.022460950538516045, -0.011689935810863972, -0.027329981327056885, 0.07293087989091873, -0.03069353476166725, 0.05429817736148834, -0.01308195199817419, 0.017668722197413445]}
Embedding shape: 10
{'x': [-0.02869587577879429, -0.0002648509689606726, -0.007186499424278736, -0.0003750955802388489, 0.012458174489438534, 0.06721009314060211, -0.013404129073023796, 0.03204648941755295, -0.021021844819188118, 0.04968355968594551]}
Embedding shape: 10
{

## Use MLTransform in read mode

In `read` mode, `MLTransform` uses the artifacts generated during `write` mode. In this case, the transform and its attributes are loaded from the saved artifacts. You don't need to specify the artifacts again during `read` mode.

In this way, `MLTransform` provides consistent preprocessing steps for training and inference workloads.

In [29]:
test_content = [
    {
        'x': 'This is a test sentence'
    },
    {
        'x': 'The park is full of dogs'
    },
    {
        'x': "Should I sign up for Medicare Part B if I have Veterans' Benefits?"
    }
]

# Uses the T5 model to generate text embeddings
with beam.Pipeline() as pipeline:
  data_pcoll = (
          pipeline
          | "CreateData" >> beam.Create(test_content))
  transformed_pcoll = (
      data_pcoll
      | "MLTransform" >> MLTransform(read_artifact_location=artifact_location_t5))

  transformed_pcoll | beam.Map(truncate_embeddings) | 'LogOutput' >> beam.Map(print)


{'x': [0.00036313451710157096, -0.03929319977760315, -0.03574873134493828, 0.05015222355723381, 0.04295048117637634, 0.04800170287489891, 0.006883862894028425, -0.02567591704428196, -0.048067063093185425, 0.036534328013658524]}
{'x': [-0.053793832659721375, 0.006730600260198116, -0.025130020454525948, 0.04363932088017464, 0.03323192894458771, 0.008803879842162132, -0.015412433072924614, 0.008926985785365105, -0.061175212264060974, 0.04573329910635948]}
{'x': [-0.03464885801076889, -0.003521254053339362, -0.010239563882350922, -0.018618224188685417, 0.004094892647117376, 0.062059689313173294, -0.013881963677704334, -0.000863900815602392, -0.029874078929424286, 0.03353121876716614]}
